In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from numpy.linalg import inv

from astropy.table import Table, Column, vstack, hstack, unique, SortedArray,SCEngine
import astropy.units as u

from astropy.io import fits, ascii 

import glob
import os

import numpy
from scipy.signal import medfilt
from scipy.interpolate import interp1d
from numba import njit
import tqdm
from pandas import DataFrame 


In [10]:
# read in high S/N table 
# loop of over the rows 
# Night and Date to make spectra file name: PETAL_LOC, NIGHT, TILEID
# load hdul
# cut r camera, "r_wavelength" 
# Grab wavelength 
# Grab corresponding spectrum 
# FIBER #

In [11]:
highSN = Table.read("/Volumes/GoogleDrive/My Drive/HighS_N.fits")   # reads in table from previous code

PETAL = (highSN['PETAL_LOC'].data)  # load each column as an array
NIGHT = (highSN['NIGHT'].data)  # load each column as an array
TILEID = (highSN['TILEID'].data)  # load each column as an array

combined = np.vstack((PETAL, TILEID,NIGHT)).T  # combines each element of each array together 
print(combined)


[[       3    63075 20200305]
 [       1    63075 20200305]
 [       6    63075 20200305]
 ...
 [       7    63075 20200219]
 [       5    63159 20200314]
 [       8    63159 20200314]]


In [12]:
tileid = []  # creates an empty array
for row in highSN:
    file = str(row['PETAL_LOC']) + '-' + str(row['TILEID']) + '-' + str(row['NIGHT'])  # goes through each element of the array and grabs the wanted elements of each one. This then combines them in the right format 
    file_tileid = str(row['TILEID']) + '/' + str(row['NIGHT']) + '/coadd-' + file  # this grabs the necessary element of each array and combines them to make part of our path in the next cell
    tileid.append(file_tileid)  # appends the file created above to them empty array
#     print(file)

In [13]:
file = ['/Volumes/GoogleDrive/My Drive/andes (1)/tiles/' + x +'.fits' for x in tileid]  # this combines all of the elements grabbed above to make a filepath


In [14]:
# for x,y in zip(list1,list2):

In [15]:
for i in range(2843,3843):

    hdul = fits.open(file[i])  # opens the fit data that belongs to the i sub_file and gets the information from that file

    r_wave = (hdul['R_WAVELENGTH'].data)  # Takes the chosen row of the hdul file
    r_flux = (hdul['R_FLUX'].data)  # Takes the chosen row of the hdul file
    r_ivar = (hdul['R_IVAR'].data)  # Takes the chosen row of the hdul file
    
    FIBER = (highSN['FIBER'].data)   # Takes the chosen row of the hdul file

    fibermap = hdul['FIBERMAP'].data  # Takes the chosen row of the hdul file
    fibers = fibermap['FIBER']   

#     print(FIBER[i])     # prints each element of FIBER 
    index = (np.where(np.in1d(fibers, FIBER[i])))   # prints which index is where fibers and FIBER matches
#     print(fibers[np.where(np.in1d(fibers, FIBER[i]))])  # plugs in the index to make sure this is where the number matches 
    index_ = list(index[0])  # converts the first element of the tuple created and converts it to a list.
#     print(index_[0])  # prints the first element of the list 
    
    
    rflux = r_flux[index_[0],:]   # plugs in the index found above and finds the matching spectrum 
    rivar = r_ivar[index_[0],:]   # plugs in the index found above and finds the matching spectrum 
    rwave = r_wave
    
    np.savez('/Volumes/GoogleDrive/My Drive/rflux.npz', rflux = rflux, overwrite = True)   # saves the multiple arrays to one file 
    np.savez('/Volumes/GoogleDrive/My Drive/rivar.npz', rivar = rivar, overwrite = True)
    np.savez('/Volumes/GoogleDrive/My Drive/rwave.npz', rwave = rwave, overwrite = True)
    
#     plt.title('Spectrum', fontsize = 15)  #  places a title and sets font size
#     plt.xlabel('Wavelength', fontsize = 15)   # places a label on the x axis and sets font size 
#     plt.ylabel('$\\mathrm{flux\\,[10^{-17}\\, erg \\, cm^{-2} \\, s^{-1} \\, \\AA^{-1}] }$', fontsize = 15) # places a label on the y axis and sets font size

#     plt.plot(r_wave, rflux)  # plots the lists we just created using a function from matplotlib.pyplot. This plots both the x and y lists.
#     plt.show() 